#Confirm your Torch version(torch 2.1.0+cu121 was validated ok)

In [ ]:
import torch
torch.__version__

# Install package

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install jedi
!pip install --pre dgl -f https://data.dgl.ai/wheels-test/cu121/repo.html
!pip install --pre dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install dgllife
!pip install rdkit-pypi
!pip install --pre deepchem
!pip install optuna
!pip install seaborn
!pip install cairosvg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.0/881.0 MB 1.9 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import deepchem as dc
import deepchem.metrics
import numpy as np
import pandas as pd
import os
import tempfile
import sys
from sklearn import metrics
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
np.set_printoptions(threshold=sys.maxsize)

Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


# 1.1 Radom seed selection

In [ ]:
import random
import torch
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [ ]:
import gc
import os
import torch

test= pd.read_csv('/content/eli_duplcate_hf.csv')
df2= pd.DataFrame(test)

rng = np.random.default_rng()
r2 = dc.metrics.Metric(dc.metrics.r2_score, np.mean, mode="regression")
test_log=[]
test_seed=[]

for i in rng.integers(1,10000, size=200):
  print(i)
  set_seed(i)
  # use 10% of datasets to perform seed selection in acceptable training time, you can change this ratio to make a more robust results
  test_df = df2.sample(frac=0.1)
  test_smiles=list(test_df['smiles'])
  test_label=list(test_df['half_life'])
  X = dc.feat.DMPNNFeaturizer(features_generators=["rdkit_desc_normalized"]).featurize(test_smiles)
  dataset = dc.data.NumpyDataset(X=X, y=test_label)
  splitter = dc.splits.splitters.RandomSplitter()
  trainset, testset = splitter.train_test_split(dataset, frac_train=0.9, seed=i)
  #release memory
  gc.collect()
  torch.cuda.empty_cache()
  #initializing a model should make a independent trial each time, your loss and r2 score should not be very high or improving gradually
  model = dc.models.DMPNNModel(mode='regression', depth=6, enc_dropout_p= 0, enc_hidden=1800, ffn_dropout_p= 0, ffn_hidden=1800, ffn_layers= 2, batch_size=50, global_features_size=200, optimizer=opt_adam, device = 'cuda')
  # higher epoch more robsut results, but training time not in linear increment. Epoch 30 is my DMPNN model start to enter plateau stage during training
  loss =  model.fit(trainset, nb_epoch=30)
  print(loss)
  train_predictions = model.evaluate(testset, metrics= r2)
  train_score = train_predictions['mean-r2_score']
  print(train_score)
  #The objective function can substitute to other metrics. e.g: MAE, RMSE, R2, accuracy ...etc
  test_log.append(train_score)
  test_seed.append(i)
  # clearing training result in case interfering next trail
  gc.collect()
  torch.cuda.empty_cache()

max_value= max(test_log)
print("Max value element : ", max_value)
index = test_log.index(max_value)
select_seed=test_seed[index]
print(select_seed)

# 1.2 Define datasets of cross-validation and testing

In [ ]:
select_seed= 1348 # Here is my best random seed

DMPNN_featurizer=dc.feat.DMPNNFeaturizer(features_generators=["rdkit_desc_normalized"])
# GCN_featurizer = dc.feat.graph_features.ConvMolFeaturizer()

set_seed(select_seed)

#loader = dc.data.data_loader.CSVLoader(tasks: =>column name of target values , feature_field: =>column name of input smiles, featurizer = DMPNN_featurizer for D-MPNN, GCN_featurizer for GCN)
loader = dc.data.data_loader.CSVLoader(tasks=['half_life'] , feature_field = "smiles", featurizer = DMPNN_featurizer)
# loader = dc.data.data_loader.CSVLoader(tasks=['half_life'] , feature_field = "smiles", featurizer = GCN_featurizer)
dataset= loader.create_dataset('/content/eli_duplcate_hf.csv')

#determine how to split and the train/ test ratio of your datasets
splitter = dc.splits.splitters.RandomSplitter()
trainset, testset = splitter.train_test_split(dataset, frac_train=0.9, seed=select_seed) # seed: 1348

# Create k-fold coss-validation datasets, in k*(train, valid) style
fold_datas = splitter.k_fold_split(trainset, k=5)

NameError: name 'dc' is not defined

# 1.3.1 Hyperparameters searching (GCN)

In [ ]:
import optuna
r2 = dc.metrics.Metric(dc.metrics.r2_score, np.mean, mode="regression")
RMSE = dc.metrics.Metric(dc.metrics.mean_squared_error, np.sqrt, mode="regression")

def opt_GCN(trial):

  model = dc.models.GraphConvModel(
                        n_tasks= 1,
                        batch_size= trial.suggest_int('batch_size',16, 512, step=16),
                        learning_rate= trial.suggest_float('learning_rate',1e-3, 1e-2,log=True),
                        # graph_conv_layers can be [64], [64,64],[64,64,64] .. etc multiple layers but need brackets
                        graph_conv_layers= trial.suggest_categorical('graph_conv_layers',[[64,64,64,64],[128,128,128,128],[64,64,64,64,64],[128,128,128,128,128]]),
                        dense_layer_size= trial.suggest_int('dense_layer_size', 256, 512, step=2),
                        dropouts= trial.suggest_float('dropouts',1e-3, 1e-2, log=True),
                        #epoch= trial.suggest_int('batch_size',0, 150, step=10),
                        batch_normalize= True,
                        mode= 'regression' )




  CV_train_score = 0
  CV_valid_score = 0

  for train_set, val_set in fold_datas:

    model.fit(train_set, nb_epoch=100, deterministic=True)
    train_predictions = model.evaluate(train_set,[r2])
    train_score = train_predictions['mean-r2_score']
    CV_train_score += train_score

    valid_predictions = model.evaluate(val_set,[r2])
    valid_score = valid_predictions['mean-r2_score']
    CV_valid_score += valid_score

    trial.report(valid_score,step=100)

    if trial.should_prune():
      print("---------------prune----------------")
      raise optuna.TrialPruned()

    #print(str(args_dict))
  print(CV_train_score/5)
  print(CV_valid_score/5)
  CV_valid_score= CV_valid_score/5

  return 1-CV_valid_score

sampler = TPESampler(**TPESampler.hyperopt_parameters())
study_gcn = optuna.create_study(direction='minimize', sampler=sampler,
               pruner=optuna.pruners.HyperbandPruner(min_resource=10, max_resource=100, reduction_factor= 5)
                            )
study_gcn.optimize(opt_GCN, n_trials = 100, n_jobs=-1)

print('Number of finished trials:', len(study_gcn.trials))
print('Best trial parameters:', study_gcn.best_trial.params)
print('Best score:', study_gcn.best_value)

In [ ]:
best_GCN_para= study_gcn.best_trial.params

# 1.3.2Hyperparameters searching (DMPNN)

In [ ]:
from deepchem.models import DMPNNModel
import numpy as np
from deepchem.models import optimizers, losses
import optuna
from optuna.samplers import TPESampler

device = 'cuda'
r2 = dc.metrics.Metric(dc.metrics.r2_score, np.mean, mode="regression")
RMSE = dc.metrics.Metric(dc.metrics.mean_squared_error, np.sqrt, mode="regression")

def opt_DMPNN(trial):

  model = dc.models.DMPNNModel(
                        mode= 'regression',
                        n_tasks= 1,
                        use_default_fdim= True,
                        enc_hidden= trial.suggest_int('enc_hidden', 300, 1800, step=100),
                        depth= trial.suggest_int('depth', 6, 10, step=2),
                        enc_dropout_p= trial.suggest_float('enc_dropout_p', 0.0001, 0.1, log= True),
                        ffn_hidden= trial.suggest_int('ffn_hidden', 300, 1800, step=100),
                        ffn_layers= trial.suggest_int('ffn_layers', 2, 10, step= 2),
                        ffn_dropout_p= trial.suggest_float('ffn_dropout_p', 0.00001, 0.1, log= True),
                        learning_rate= trial.suggest_float('learning_rate',0.00001, 0.1, log=True),
                        batch_size= trial.suggest_int('batch_size', 30, 100, step=10),
                        global_features_size=200,
                        batch_normalize=True,
                        opt_adam = optimizers.AdamW(learning_rate= 'learning_rate', weight_decay=trial.suggest_float('weight_decay', 0.00001, 0.1, log= True)),
                        device = device
                                                                      )


  step=40
  CV_train_RMSE = 0
  CV_valid_RMSE = 0
  count=0
  for i in range(len(fold_datas)):
    opt_t, opt_v= fold_datas[i]
    model.fit(opt_t, nb_epoch=step)
    train_predictions_RMSE = model.evaluate(opt_t,[RMSE])
    train_RMSE= train_predictions_RMSE['sqrt-mean_squared_error']
    print('train_loss: ', train_RMSE)
    CV_train_RMSE += train_RMSE

    valid_predictions_RMSE = model.evaluate(opt_v,[RMSE])
    valid_RMSE= valid_predictions_RMSE['sqrt-mean_squared_error']
    print('valid_loss: ', valid_RMSE)
    CV_valid_RMSE += valid_RMSE

    count+=1
    if count > 2:
      trial.report(valid_RMSE/3,step=step)

      if trial.should_prune():
        print("---------------prune----------------")
        raise optuna.TrialPruned()


  print('Trail end, final train loss: ', CV_train_RMSE/5)
  print('Trail end, final valid loss: ',CV_valid_RMSE/5)

  CV_final_RMSE= CV_valid_RMSE/5
  print(CV_final_RMSE)

  return CV_final_RMSE

sampler = TPESampler(**TPESampler.hyperopt_parameters())
study_dmpnn = optuna.create_study(direction='minimize',sampler=sampler,
               pruner=optuna.pruners.HyperbandPruner(min_resource=6, max_resource=40, reduction_factor= 5))

study_dmpnn.optimize(opt_DMPNN, n_trials = 200, n_jobs=-1)

print('Number of finished trials:', len(study_dmpnn.trials))
print('Best trial parameters:', study_dmpnn.best_trial.params)
print('Best score:', study_dmpnn.best_value)

In [ ]:
best_DMPNN_para= study_dmpnn.best_trial.params

# 1.3.3 Plots of Optuna Trial (DMPNN for an example)

In [ ]:
fig = optuna.visualization.plot_param_importances(study_dmpnn)
fig.show()

NameError: name 'optuna' is not defined

In [ ]:
fig2= optuna.visualization.plot_edf(study_dmpnn)
fig2.show()

NameError: name 'optuna' is not defined

# 1.4 Define model used in training (GCN or DMPNN, only choosing 1 for training)

In [ ]:
# GraphConvModel is a keras based model
model_gcn = dc.models.GraphConvModel(
                        n_tasks= 1,
                        batch_size= best_GCN_para['batch_size'],
                        learning_rate= best_GCN_para['learning_rate'],
                        graph_conv_layers= best_GCN_para['graph_conv_layers'],
                        dense_layer_size= best_GCN_para['dense_layer_size'],
                        dropouts= best_GCN_para['dropouts'],
                        batch_normalize= True,
                        mode= 'regression' )


In [ ]:
from deepchem.models import optimizers, losses

set_seed(select_seed)
opt_adam = optimizers.AdamW(best_DMPNN_para['learning_rate'], weight_decay=best_DMPNN_para['weight_decay'])

# DMPNNModel is a pytorch based model
model_dmpnn = dc.models.DMPNNModel(
                        mode= 'regression',
                        n_tasks= 1,
                        enc_hidden= best_DMPNN_para['enc_hidden'],
                        depth= best_DMPNN_para['depth'],
                        enc_dropout_p= best_DMPNN_para['enc_dropout_p'],
                        ffn_hidden= best_DMPNN_para['ffn_hidden'],
                        ffn_layers= best_DMPNN_para['ffn_layers'],
                        ffn_dropout_p= best_DMPNN_para['ffn_dropout_p'],
                        learning_rate= best_DMPNN_para['learning_rate'],
                        batch_size= best_DMPNN_para['batch_size'],
                        global_features_size=200,
                        optimizer=opt_adam,
                        device = 'cuda' )

# 1.5 Main function of 5-fold cv training and testing with record

In [ ]:
import pandas as pd
import numpy as np
import deepchem as dc

r2 = dc.metrics.Metric(dc.metrics.r2_score, np.mean, mode="regression")
RMSE = dc.metrics.Metric(dc.metrics.mean_squared_error, np.sqrt, mode="regression")

def main(start_epoch:int, end_epoch:int, fold_datas:deepchem.data.DiskDataset, testset:deepchem.data.DiskDataset, model:deepchem.models, ckpt_dir, best_dir):

  train_history3=[]
  valid_historty3=[]
  train_loss3=[]
  valid_loss3=[]
  test_history3=[]
  test_loss3=[]
  train_P=[]
  valid_P=[]
  test_P=[]
  # Threshold of determining best checkpoint by RMSE
  best=1

  for i in range(start_epoch, end_epoch+1):
    CV_train_score = 0
    CV_valid_score = 0
    CV_train_RMSE = 0
    CV_valid_RMSE = 0
    test_o_score = 0
    test_o_RMSE = 0

    for train_set, val_set in fold_datas:

      loss = model.fit(train_set, nb_epoch = i)

      train_predictions = model.evaluate(train_set, metrics= r2)
      train_score = train_predictions['mean-r2_score']
      CV_train_score += train_score
      valid_predictions = model.evaluate(val_set, metrics= r2)
      valid_score = valid_predictions['mean-r2_score']
      CV_valid_score += valid_score
      test_predictions = model.evaluate(testset, metrics= r2)
      test_score = test_predictions['mean-r2_score']
      print('test_r2: ', test_score)
      test_o_score += test_score
      #rmse
      train_predictions_RMSE = model.evaluate(train_set,[RMSE])
      rse= train_predictions_RMSE['sqrt-mean_squared_error']
      train_RMSE = rse
      CV_train_RMSE += train_RMSE
      valid_predictions_RMSE = model.evaluate(val_set,[RMSE])
      rse_val= valid_predictions_RMSE['sqrt-mean_squared_error']
      valid_RMSE = rse_val
      CV_valid_RMSE += valid_RMSE
      test_RMSE = model.evaluate(testset,[RMSE])
      res_test = test_RMSE['sqrt-mean_squared_error']
      test_RMSE_score = res_test
      test_o_RMSE += test_RMSE_score
    # collecting performace records
    tp= model.predict(train_set)
    vp= model.predict(val_set)
    tep= model.predict(testset)
    train_P.append(tp)
    valid_P.append(vp)
    test_P.append(tep)
    # you can add std error function here for more complete results, before get the mean performance of each epoch
    CV_valid_score = CV_valid_score/5
    CV_train_score = CV_train_score/5
    CV_train_RMSE = CV_train_RMSE/5
    CV_valid_RMSE = CV_valid_RMSE/5
    test_o_score = test_o_score/5
    test_o_RMSE = test_o_RMSE/5

    print('--------epoch--------:',i)
    print('train_r2: ', CV_train_score)
    print('train_loss: ', CV_train_RMSE)
    print('valid_r2: ', CV_valid_score)
    print('valid_loss: ', CV_valid_RMSE)
    print('test_r2: ', test_o_score)
    print('test_loss: ', test_o_RMSE)

    train_history3.append(CV_train_score)
    valid_historty3.append(CV_valid_score)
    train_loss3.append(CV_train_RMSE)
    valid_loss3.append(CV_valid_RMSE)
    test_history3.append(test_o_score)
    test_loss3.append(test_o_RMSE)

    model.save_checkpoint(model_dir=f'{ckpt_dir}')
    #save the checkpoint of best result, you can use model.restore('path to checkpoint') to load a trained model
    if test_o_RMSE < best:
      model.save_checkpoint(max_checkpoints_to_keep=1, model_dir=f'{best_dir}')
      best = test_o_RMSE
    train_log= pd.DataFrame.from_dict({'train_r2': np.array(train_history3), 'train_rmse': np.array(train_loss3)})
    tpd= np.array(train_P)
    # This function can prevent unexpectable termination of your training, you can load this file to plot training curve
    np.save('tpd.npy', tpd)
    # This function can prevent unexpectable termination of your training,
    # you can svae this file to evaluate performance of your most recently trained model
    train_log.to_csv('train_log.csv', index=False)
    # validation and testing are same as trainimg
    valid_log= pd.DataFrame.from_dict({'valid_r2': np.array(valid_historty3), 'valid_rmse': np.array(valid_loss3)})
    vpd= np.array(valid_P)
    np.save('vpd.npy', vpd)
    valid_log.to_csv('valid_log.csv', index=False)
    test_log= pd.DataFrame.from_dict({'test_r2': np.array(test_history3), 'test_rmse': np.array(test_loss3)})
    tepd= np.array(test_P)
    np.save('tepd.npy', tepd)
    test_log.to_csv('test_log.csv', index=False)

  return train_log, valid_log, test_log, [tpd, vpd, tepd]

In [ ]:
#Demonstrate DMPNN here

!mkdir /content/dmpnn
!mkdir /content/dmpnn/ckpt_dir
!mkdir /content/dmpnn/best_dir

train, valid, test, pred = main(start_epoch=1, end_epoch=50, fold_datas=fold_datas, testset=testset, model=model_dmpnn, ckpt_dir='/content/dmpnn/ckpt_dir', best_dir='/content/dmpnn/best_dir')

# 1.6 Save and output results

In [ ]:
def save_table(train_df: pd.DataFrame, valid_df: pd.DataFrame, test_df: pd.DataFrame, predict_arr: np.array, testset:deepchem.data.DiskDataset, out_name:str, out_path:str):
  #Objective: r2
  ind_M= np.argmax(test_df['test_r2'])
  print(ind_M)
  print(train_df['train_r2'][ind_M])
  print(valid_df['valid_r2'][ind_M])
  print(test_df['test_r2'][ind_M])
  print('\n')
  print(train_df['train_rmse'][ind_M])
  print(valid_df['valid_rmse'][ind_M])
  print(test_df['test_rmse'][ind_M])
  print('-----------------------------')
  #Objective: rmse
  ind_M2= np.argmin(test_df['test_rmse'])
  print(ind_M2)
  print(train_df['train_r2'][ind_M2])
  print(valid_df['valid_r2'][ind_M2])
  print(test_df['test_r2'][ind_M2])
  print('\n')
  print(train_df['train_rmse'][ind_M2])
  print(valid_df['valid_rmse'][ind_M2])
  print(test_df['test_rmse'][ind_M2])
  # Here the rmse reuslts are chosen to output, ypu can change to r2 by substituting ind_M2 to ind_M
  # the target values are in hours and taken nature log in original datasets, but we transfom preidcted values to minutes for evalute the real performance
  exp_pred_te=np.exp(np.ravel(predict_arr[2][ind_M2]))
  te_pred_P= 60*np.array((exp_pred_te))
  exp_test_te=np.exp(np.ravel(testset.y))
  te_pred_T= 60*np.array(exp_test_te)

  #record of prediction results to testset
  df_r3=pd.DataFrame()
  df_r3['SMILES']=testset.ids
  df_r3['Observed(min)']=te_pred_T
  df_r3['Prediction of testing-set(min)']=te_pred_P
  print(df_r3)
  df_r3.to_csv(f"{out_path}/{out_name}_DMPNN_testing_pred_vs_obsered.csv", index=False)

  df_r3['difference']= np.abs(df_r3['Prediction of testing-set(min)']- df_r3['Observed(min)'])
  print(df_r3[df_r3['difference']>60])
  drawsmi=df_r3[df_r3['difference']>60]['SMILES']
  bigger60= df_r3[df_r3['difference']>60]
  # 60 mins is a threshold in this study, if the differencebetwwen observed and prediction bigger than 60 mins then it would be considered as bad prediction;
  # smaller than 60 mins are considred acceptable
  bigger60.to_csv(f'{out_path}/{out_name}_biggerthan60.csv', index=False)

  df_r4=pd.DataFrame()
  # training record
  df_r4['fold_train_history']=train_df['train_r2']
  df_r4['fold_valid_history']=valid_df['valid_r2']
  df_r4['fold_test_history']=test_df['test_r2']
  df_r4['fold_train_loss']=train_df['train_rmse']
  df_r4['fold_valid_loss']=valid_df['valid_rmse']
  df_r4['fold_test_loss']=test_df['test_rmse']

  df_r4.to_csv(f"{out_path}/{out_name}_deepchem_record.csv")

In [ ]:
# take DMPNN model as example continued here
save_table(train, valid, test, pred, testset, out_name='No_dataselection', out_path='/content/dmpnn')

# 1.7 Prediction functions

In [ ]:
from deepchem.models import optimizers, losses

select_seed= 1348
set_seed(select_seed)
# This model considered as your trained model in previous parts, this seed and model are best hyperparameters combination of my DMPNN model
weight_decay = 5 # also known as l2_regularization_lambda
learning_rate = 4
opt_adam = optimizers.AdamW(learning_rate= 0.0001, weight_decay=0.00001)


model_dmpnn = dc.models.DMPNNModel(    mode='regression',
                    depth=6,
                    enc_dropout_p= 0,
                    enc_hidden=1800,
                    ffn_dropout_p= 0,
                    ffn_hidden=1800,
                    ffn_layers= 2,
                    batch_size=50,
                    global_features_size=200,
                    optimizer=opt_adam,
                    device = 'cuda'    )

Random seed set as 1348


In [ ]:
# read unseen smiles, and converting to graphs
import pandas as pd
import numpy as np

# ***You need to run the training steps first to train a model, then you can load checkpint with model.restore() to predict unseen compound
def model_prediction(unseen_in_csv, featurizer, model, checkpoint):
  df=pd.read_csv(f'{unseen_in_csv}')
  # your csv file needs to have one column named 'smiles'
  loader_unseen = dc.data.data_loader.CSVLoader(tasks=[], feature_field = "smiles", featurizer = featurizer)
  # loader_unseen = dc.data.data_loader.CSVLoader(tasks=[], feature_field = "smiles", featurizer = DMPNN_featurizer)
  # loader_unseen = dc.data.data_loader.CSVLoader(tasks=[] , feature_field = "smiles", featurizer = GCN_featurizer)
  dataset_test= loader_unseen.create_dataset(f'{unseen_in_csv}')
  model.restore(f'{checkpoint}')
  predicted_halflife= model.predict(dataset_test)
  #exp_pred_te=np.exp(np.ravel(predicted_halflife))
  #te_pred_P= 60*np.array((exp_pred_te))
  output_df= pd.DataFrame()
  output_df['smiles']= df['smiles']
  output_df['predicted_halflife']= np.array(predicted_halflife)
  #output_df['predicted_halflife']= te_pred_P
  print(output_df)
  output_df.to_csv('prediction_result.cav')



DMPNN_featurizer=dc.feat.DMPNNFeaturizer(features_generators=["rdkit_desc_normalized"])
model_prediction('/content/test_smiles.csv', DMPNN_featurizer, model_dmpnn,'/content/checkpoint1.pt')



                                               smiles  predicted_halflife
0   [2H]C([2H])(c1cc(Cl)ccc1NCc1ccc(S(C)(=O)=O)cc1...           -3.429336
1   CC(=O)N[C@@H]1CC[C@@]2(C)C(=CC[C@H]3[C@@H]4CC=...           -3.852561
2   COc1cncc(C2=CC[C@H]3[C@@H]4CC=C5C[C@H](NC(=O)C...           -4.256824
3   CC(=O)N[C@@H]1CC[C@@]2(C)C(=CC[C@H]3[C@@H]4CC=...           -4.391540
4   COc1cncc(C2=CC[C@H]3[C@@H]4CCC5C[C@@H](NC(C)=O...           -3.914470
5   CC(C)C(=O)N[C@H]1CC[C@@]2(C)C(CC[C@H]3[C@@H]4C...           -3.928624
6   CCNC(=O)N[C@@H]1CC[C@@]2(C)C(=CC[C@H]3[C@@H]4C...           -4.332470
7   C[C@]12CC[C@H]3[C@@H](CC=C4C[C@H](NC(=O)C5CCC5...           -4.213406
8   C[C@]12CC[C@H]3[C@@H](CCC4C[C@H](NC(=O)C5CC5)C...           -4.089286
9   CCCC(=O)N[C@@H]1CC[C@@]2(C)C(=CC[C@H]3[C@@H]4C...           -4.377962
10  C[C@]12CC[C@H]3[C@@H](CC=C4C[C@H](NC(=O)C5CC5)...           -4.212689
11  C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)NCC[C@@]43C)[...           -3.974460


# 1.8 Plot functions

In [ ]:
import matplotlib.pyplot as plt
from google.colab import files
def plot_r2(epoch, train_df, valid_df, test_df, filename, file_path):
  plt.xlabel("Epoch")
  plt.ylabel("r2_score")
  x = range(1,epoch+1)
  y1= train_df['train_r2']
  y2= valid_df['valid_r2']
  y3= test_df['test_r2']
  g = plt.plot(x, y1, color = "saddlebrown", label = "train_r2")
  g = plt.plot(x, y3, color = "sandybrown", label = "valid_q2")
  g = plt.plot(x, y2, color = "red", label = "test_r2")
  plt.ylim(bottom = 0, top = 1)
  plt.legend(loc = "lower right")
  plt.savefig(f"{file_path}/{filename}_r2.png")
  plt.show()

In [ ]:
def plot_rmse(epoch, train_df, valid_df, test_df, filename, file_path):
  plt.xlabel("Epoch")
  plt.ylabel("RMSE")
  x = range(1,epoch+1)
  y1= train_df['train_rmse']
  y2= valid_df['valid_rmse']
  y3= test_df['test_rmse']
  g = plt.plot(x, y1, color = "saddlebrown", label = "train_RMSE")
  g = plt.plot(x, y2, color = "sandybrown", label = "valid_RMSE")
  g = plt.plot(x, y3, color = "red", label = "test_RMSE")
  plt.ylim(bottom = 0, top = 1)
  plt.legend(loc = "lower right")
  plt.savefig(f"{file_path}/{filename}_rmse.png")
  plt.show()

In [ ]:
!mkdir /content/dmpnn/result_plots

plot_r2(epoch=50, train_df= train_sus, valid_df=valid_sus, test_df=test_sus, filename='dmpnn_result_r2', file_path='/content/dmpnn/result_plots')
plot_rmse(epoch=50, train_df= train_sus, valid_df=valid_sus, test_df=test_sus, filename='dmpnn_result_rmse', file_path='/content/dmpnn/result_plots')

In [ ]:
import seaborn as sns

# Demonstration of plotting residual plot using observed and predicted values
df_r3=pd.read_csv('No_dataselection_DMPNN_testing_pred_vs_obsered.csv')
sns.residplot(df_r3, x="Observed(min)", y="Prediction of testing-set(min)", lowess=True, line_kws=dict(color='r'))

# 1.9 Classifcation results of trained model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt

def result_in_class(pred_result_file, observed_col, predict_col):

  result= pd.read_csv(f'{pred_result_file}')
  ytest_O= np.array(result[f'{observed_col}'])
  ytest_P= np.array(result[f'{predict_col}'])

  obs=[]
  pred=[]

  for i in range(len(ytest_P)):
    #print(i)
    if float(ytest_P[i]) < 30.0:
      pred.append(0)
    if float(ytest_O[i]) < 30.0:
      obs.append(0)
    if 30.0 <= float(ytest_P[i]) <= 60.0:
      pred.append(1)
    if 30.0 <= float(ytest_O[i]) <= 60.0:
      obs.append(1)
    if float(ytest_P[i]) > 60.0:
      pred.append(2)
    if float(ytest_O[i]) > 60.0:
      obs.append(2)
    else:
      print('NULL values')
      continue

  pred= np.array(pred)
  obs= np.array(obs)

  plt.figure(figsize=(10,6))
  fx=sns.heatmap(confusion_matrix(obs, pred), annot=True, fmt=".2f",cmap="GnBu")
  fx.set_title('Confusion Matrix \n');
  fx.set_xlabel('\n Predicted class\n')
  fx.set_ylabel('Actual class\n');
  fx.xaxis.set_ticklabels(['unstable','moderate','stable'])
  fx.yaxis.set_ticklabels(['unstable','moderate','stable'])
  plt.show()
  print('\n')
  target_name= ['unstable','moderate','stable']
  print(classification_report(obs, pred, target_names= target_name))
  print('\n')
  print('mcc: ', matthews_corrcoef(obs, pred))

In [ ]:
result_in_class('/content/deepchem_DMPNN_testing_result_record.csv', 'Observed(min)', 'Prediction of testing-set(min)')

# 1.10 Plot molecules

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

#needed for show_mols
#more: http://rdkit.blogspot.com/2020/04/new-drawing-options-in-202003-release.html
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
import cairosvg
import math
import os

from rdkit import rdBase

In [ ]:
import math
import cairosvg
#this functon is designed to plot a group of molecules in defined number per row
def show_mols(mols, mols_per_row = 5, size=300, min_font_size=12, legends=[], file_name=''):

  if legends and len(legends) < len(mols):
    print('legends is too short')
    return None

  mols_per_row = min(len(mols), mols_per_row)
  rows = math.ceil(len(mols)/mols_per_row)
  d2d = rdMolDraw2D.MolDraw2DSVG(mols_per_row*size,rows*size,size,size)
  d2d.drawOptions().minFontSize = min_font_size
  if legends:
    d2d.DrawMolecules(mols, legends=legends)
  else:
    d2d.DrawMolecules(mols)
  d2d.FinishDrawing()

  if file_name:
    with open('d2d.svg', 'w') as f:
      f.write(d2d.GetDrawingText())
      if 'pdf' in file_name:
        cairosvg.svg2pdf(url='d2d.svg', write_to=file_name)
      else:
        cairosvg.svg2png(url='d2d.svg', write_to=file_name)
      os.remove('d2d.svg')

  return SVG(d2d.GetDrawingText())

# mols_per_row and chunk_size must be matched
def divide_chunks(total_length, chunk_size):
  # looping till length l
  for i in range(0, len(total_length), chunk_size):
    yield total_length[i:i + chunk_size]


In [ ]:
df_bg60= pd.read_csv('No_dataselection_biggerthan60.csv')
drawsmi= df_bg60['SMILES']
print(drawsmi)
mols = [Chem.MolFromSmiles(x) for x in drawsmi]
legends= [str(i) for i in range(len(drawsmi))]
#print(legends)

n = 5
x = list(divide_chunks(mols, n))
y = list(divide_chunks(legends, n))

file_path= '/content/dmpnn/result_plots'

for chunck in range(len(x)):
  show_mols(mols= x[chunck], legends=y[chunck], file_name=f'{file_path}/{chunck}.png')